# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
!pip uninstall -y torch torchvision torchaudio tensorflow transformers -q
!pip install torch==2.0.0 torchvision==0.15.0 torchaudio==2.0.0 --index-url https://download.pytorch.org/whl/cpu -q
!pip install transformers==4.30.0 scikit-learn pandas numpy matplotlib seaborn tqdm -q
!pip install torch transformers scikit-learn pandas numpy -q

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import logging
import re
import warnings
warnings.filterwarnings('ignore')

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")
print(f"Версия PyTorch: {torch.__version__}")

Используемое устройство: cpu
Версия PyTorch: 2.0.0+cpu


## Подготовка

In [4]:
# Загрузка данных
df = pd.read_csv('/datasets/toxic_comments.csv',index_col=0)
print(f"Размер датасета: {df.shape}")
print(f"Распределение классов:\n{df['toxic'].value_counts()}")
print(df.head())

Размер датасета: (159292, 2)
Распределение классов:
0    143106
1     16186
Name: toxic, dtype: int64
                                                text  toxic
0  Explanation\nWhy the edits made under my usern...      0
1  D'aww! He matches this background colour I'm s...      0
2  Hey man, I'm really not trying to edit war. It...      0
3  "\nMore\nI can't make any real suggestions on ...      0
4  You, sir, are my hero. Any chance you remember...      0


Вывод: Имеется датасет с 143 тыс положительных и 16 отрицательных отзывов. Так как я собираюсь использовать bert то не проводил лематизацию и баланс класов ответов.

## Обучение

In [6]:
# Проверяем доступность устройств
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

def clean_text(text):
    """Очистка текста"""
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)  
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Загрузка только Toxic BERT
def load_toxic_bert():
    model_name = 'unitary/toxic-bert'
    try:
        print(f"Загружаю {model_name}...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModel.from_pretrained(model_name)
        print(f"✓ Успешно загружена модель: {model_name}")
        return tokenizer, model, model_name
    except Exception as e:
        raise Exception(f"Не удалось загрузить Toxic BERT: {e}")

# Загружаем модель
tokenizer, model, model_name = load_toxic_bert()
model.to(device)
model.eval()
print(f"Модель перемещена на {device}")

def get_toxic_bert_embeddings(texts, batch_size=16):  
    """Получение эмбеддингов из Toxic BERT"""
    all_embeddings = []
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        
        # Токенизация
        encoded = tokenizer.batch_encode_plus(
            batch_texts,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
        )
        
        # Переносим на нужное устройство
        encoded = {k: v.to(device) for k, v in encoded.items()}
        
        with torch.no_grad():
            outputs = model(**encoded)
        
        # Извлечение [CLS] токена
        cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        all_embeddings.extend(cls_embeddings)
    
    return np.array(all_embeddings)

def calculate_class_weights(labels):
    """Вычисление весов классов"""
    unique_classes = np.unique(labels)
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=unique_classes,
        y=labels
    )
    
    class_weights = torch.tensor(class_weights, dtype=torch.float32)
    print(f"Веса классов: {dict(zip(unique_classes, class_weights.numpy()))}")
    return class_weights

# Основной код
def main():
    # Предполагаем, что df уже определен
    print("Размер датасета:", len(df))
    print("Колонки:", df.columns.tolist())
    
    if 'text' not in df.columns:
        raise ValueError("Колонка 'text' не найдена в датафрейме")
    
    # Уменьшаем размер выборки
    sample_size = min(1000, len(df))
    sampled_df = df.sample(sample_size, random_state=42)
    
    # Очистка текста
    sampled_df['text_clean'] = sampled_df['text'].apply(clean_text)
    print(f"Очищено {len(sampled_df)} текстов")
    
    # Получение эмбеддингов
    texts = sampled_df['text_clean'].tolist()
    embeddings = get_toxic_bert_embeddings(texts)
    print(f"Размерность эмбеддингов: {embeddings.shape}")
    
    # Проверяем наличие меток toxic
    results = {
        'embeddings': embeddings,
        'sampled_df': sampled_df
    }
    
    if 'toxic' in sampled_df.columns:
        labels = sampled_df['toxic'].values
        
        # Анализ дисбаланса
        unique, counts = np.unique(labels, return_counts=True)
        print(f"Распределение классов: {dict(zip(unique, counts))}")
        
        # Вычисление весов классов
        class_weights = calculate_class_weights(labels)
        results['class_weights'] = class_weights
        
        # Разделение на train/test
        X_train, X_test, y_train, y_test = train_test_split(
            embeddings, labels, test_size=0.2, random_state=42, stratify=labels
        )
        
        print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")
        results.update({
            'X_train': X_train,
            'X_test': X_test,
            'y_train': y_train,
            'y_test': y_test
        })
    
    return results

# Запуск
if __name__ == "__main__":
    results = main()
    
    # Теперь можно получить отдельные переменные из results
    embeddings = results['embeddings']
    sampled_df = results['sampled_df']
    
    print("✓ Готово! Эмбеддинги созданы успешно")
    print(f"Размер эмбеддингов: {embeddings.shape}")
    
    # Разделение на обучающую и тестовую выборки
    if 'toxic' in sampled_df.columns:
        X_train, X_test, y_train, y_test = train_test_split(
            embeddings, 
            sampled_df['toxic'].values, 
            test_size=0.2, 
            random_state=42,
            stratify=sampled_df['toxic'].values
        )

        print(f"Обучающая выборка: {X_train.shape}")
        print(f"Тестовая выборка: {X_test.shape}")
        print(f"Метки обучающей выборки: {y_train.shape}")
        print(f"Метки тестовой выборки: {y_test.shape}")
        
        # Анализ распределения классов
        print("\nРаспределение классов в обучающей выборке:")
        unique_train, counts_train = np.unique(y_train, return_counts=True)
        print(dict(zip(unique_train, counts_train)))
        
        print("Распределение классов в тестовой выборке:")
        unique_test, counts_test = np.unique(y_test, return_counts=True)
        print(dict(zip(unique_test, counts_test)))
        
        # Сохраняем разделенные данные в results
        results.update({
            'X_train': X_train,
            'X_test': X_test,
            'y_train': y_train,
            'y_test': y_test
        })
    else:
        print("Колонка 'toxic' не найдена для разделения данных")

Используемое устройство: cpu
Загружаю unitary/toxic-bert...


Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


✓ Успешно загружена модель: unitary/toxic-bert
Модель перемещена на cpu
Размер датасета: 159292
Колонки: ['text', 'toxic']
Очищено 1000 текстов
Размерность эмбеддингов: (1000, 768)
Распределение классов: {0: 894, 1: 106}
Веса классов: {0: 0.5592841, 1: 4.716981}
Train size: 800, Test size: 200
✓ Готово! Эмбеддинги созданы успешно
Размер эмбеддингов: (1000, 768)
Обучающая выборка: (800, 768)
Тестовая выборка: (200, 768)
Метки обучающей выборки: (800,)
Метки тестовой выборки: (200,)

Распределение классов в обучающей выборке:
{0: 715, 1: 85}
Распределение классов в тестовой выборке:
{0: 179, 1: 21}


In [7]:
# Разделение на обучающую и тестовую выборки
#X_train, X_test, y_train, y_test = train_test_split(
    #embeddings, 
    #sampled_df['toxic'].values, 
    #test_size=0.2, 
   # random_state=42,
   # stratify=sampled_df['toxic'].values
#)

print(f"Обучающая выборка: {X_train.shape}")
print(f"Тестовая выборка: {X_test.shape}")

Обучающая выборка: (800, 768)
Тестовая выборка: (200, 768)


In [8]:
# 1. Логистическая регрессияв
print("Подбор параметров для Logistic Regression...")
log_reg_param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'class_weight': [None, 'balanced'],
    'solver': ['liblinear', 'saga']
}

log_reg_grid = GridSearchCV(
    LogisticRegression(random_state=42, max_iter=1000),
    log_reg_param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=0
)
log_reg_grid.fit(X_train, y_train)
best_log_reg = log_reg_grid.best_estimator_

# 2. Random Forest 
print("Подбор параметров для Random Forest...")
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'class_weight': [None, 'balanced']
}

rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    rf_param_grid,
    cv=3,  
    scoring='f1',
    n_jobs=-1,
    verbose=0
)
rf_grid.fit(X_train, y_train)
best_rf = rf_grid.best_estimator_

#3 SVM
print("Подбор параметров для SVM...")
svm_param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'],
    'class_weight': [None, 'balanced']
}

svm_grid = GridSearchCV(
    SVC(random_state=42, probability=True),
    svm_param_grid,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    verbose=0
)
svm_grid.fit(X_train, y_train)
best_svm = svm_grid.best_estimator_


# Лучшие параметры для каждой модели
print(f"\nЛучшие параметры Logistic Regression: {log_reg_grid.best_params_}")
print(f"Лучший F1-score (CV): {log_reg_grid.best_score_:.4f}")

print(f"\nЛучшие параметры Random Forest: {rf_grid.best_params_}")
print(f"Лучший F1-score (CV): {rf_grid.best_score_:.4f}")

print(f"\nЛучшие параметры SVM: {svm_grid.best_params_}")
print(f"Лучший F1-score (CV): {svm_grid.best_score_:.4f}")

# Сравнение F1-score на кросс-валидации
print("\n" + "="*60)
print("СРАВНЕНИЕ F1-SCORE НА КРОСС-ВАЛИДАЦИИ")
print("="*60)

cv_scores = {
    'Logistic Regression': log_reg_grid.best_score_,
    'Random Forest': rf_grid.best_score_,
    'SVM': svm_grid.best_score_
}

for model_name, score in cv_scores.items():
    print(f"{model_name}: F1-score = {score:.4f}")

# Определение лучшей модели по кросс-валидации
best_model_name = max(cv_scores, key=cv_scores.get)
best_cv_score = cv_scores[best_model_name]
print(f"\nЛучшая модель на валидации: {best_model_name} (F1-score: {best_cv_score:.4f})")

Подбор параметров для Logistic Regression...
Подбор параметров для Random Forest...
Подбор параметров для SVM...

Лучшие параметры Logistic Regression: {'C': 0.01, 'class_weight': None, 'solver': 'liblinear'}
Лучший F1-score (CV): 0.9700

Лучшие параметры Random Forest: {'class_weight': 'balanced', 'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 200}
Лучший F1-score (CV): 0.9584

Лучшие параметры SVM: {'C': 10, 'class_weight': None, 'gamma': 'scale', 'kernel': 'rbf'}
Лучший F1-score (CV): 0.9597

СРАВНЕНИЕ F1-SCORE НА КРОСС-ВАЛИДАЦИИ
Logistic Regression: F1-score = 0.9700
Random Forest: F1-score = 0.9584
SVM: F1-score = 0.9597

Лучшая модель на валидации: Logistic Regression (F1-score: 0.9700)


In [9]:
if best_model_name == 'Logistic Regression':
    best_model = best_log_reg
elif best_model_name == 'Random Forest':
    best_model = best_rf
else:
    best_model = best_svm

# Предсказания на тестовых данных
y_pred_test = best_model.predict(X_test)
y_pred_proba_test = best_model.predict_proba(X_test)

f1_test = f1_score(y_test, y_pred_test)


print(f"Лучшая модель: {best_model_name}")
print(f"F1-score на тестовых данных : {f1_test:.4f}")


Лучшая модель: Logistic Regression
F1-score на тестовых данных : 0.8947


Вывод: Настроили toxic-bert эмбединги на подготовленых и очищенных данных, для ускорения работы выбрали 1000 строк. Обучили три модели логистическую регрессию, случайный лес и опорные вектора. При помощи крос валидации произвели подбор гиперпаратетров и оценили f1 на тренировочных данных. Лучшей моделью показала себя модельл логистической регрессиив с f1 0.97 на валидации и 0.894 тестовых данных учитывая вес изза дисбаланса класов

## Выводы

In [11]:
# Обучение на всех данных с лучшей моделью
final_model = best_model  

# Переобучение на всех данных
final_model.fit(embeddings, sampled_df['toxic'].values)

# Функция для предсказания новых комментариев с использованием Toxic BERT
def predict_toxicity(text, model, tokenizer, bert_model):
    # Очистка текста
    cleaned_text = clean_text(text)
    
    # Токенизация
    encoded = tokenizer.encode_plus(
        cleaned_text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    # Переносим на нужное устройство
    encoded = {k: v.to(device) for k, v in encoded.items()}
    
    with torch.no_grad():
        outputs = bert_model(**encoded)
    
    # Получение эмбеддинга
    cls_embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    
    # Предсказание
    prediction = model.predict(cls_embedding)
    
    # Для моделей, которые поддерживают predict_proba
    if hasattr(model, 'predict_proba'):
        probability = model.predict_proba(cls_embedding)[0, 1]
    else:
        # Для моделей без predict_proba (например, SVM)
        probability = model.decision_function(cls_embedding)[0]
        # Преобразуем в вероятность от 0 до 1
        probability = 1 / (1 + np.exp(-probability))
    
    return prediction[0], probability

# Используем уже загруженный Toxic BERT (не нужно загружать заново)
# bert_model уже загружен ранее как model

# Примеры для тестирования
test_examples = [
    "This is a great article, thanks for sharing!",
    "You are so stupid and worthless, I hate you!",
    "I completely disagree with your opinion on this matter.",
    "Go die in a hole, nobody wants you here!",
    "Interesting perspective, I never thought about it that way.",
    "Your mother should have aborted you, you piece of trash!",
    "Could you please provide more details about this topic?",
    "I hope you get cancer and suffer for eternity!",
    "The weather is really nice today, isn't it?",
    "You're such an idiot, your opinion doesn't matter at all!"
]

# Тестирование на примерах
print("\nРезультаты предсказаний (с использованием Toxic BERT):")
print("-" * 80)
for i, example in enumerate(test_examples, 1):
    prediction, probability = predict_toxicity(example, final_model, tokenizer, model)
    toxicity_label = "ТОКСИЧНЫЙ" if prediction == 1 else "НЕ ТОКСИЧНЫЙ"
    color = "\033[91m" if prediction == 1 else "\033[92m"  # Красный для токсичного, зеленый для нетоксичного
    reset_color = "\033[0m"
    
    print(f"{i}. {example}")
    print(f"   → Предсказание: {color}{toxicity_label}{reset_color}")
    print(f"   → Вероятность токсичности: {probability:.4f}")
    
    # Определение уровня уверенности
    if probability > 0.8 or probability < 0.2:
        confidence = "Высокая"
    elif probability > 0.6 or probability < 0.4:
        confidence = "Средняя"
    else:
        confidence = "Низкая"
    
    print(f"   → Уверенность: {confidence}")
    print("-" * 80)

# Дополнительная информация о финальной модели
print(f"\nИспользуемая финальная модель: {type(final_model).__name__}")
print(f"Используемая BERT модель: {model_name}")
print(f"Параметры модели: {final_model.get_params() if hasattr(final_model, 'get_params') else 'N/A'}")

# Функция для пакетного предсказания
def predict_toxicity_batch(texts, model, tokenizer, bert_model, batch_size=8):
    """Предсказание токсичности для списка текстов"""
    predictions = []
    probabilities = []
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_embeddings = []
        
        for text in batch_texts:
            cleaned_text = clean_text(text)
            
            # Токенизация
            encoded = tokenizer.encode_plus(
                cleaned_text,
                add_special_tokens=True,
                max_length=128,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt'
            )
            
            encoded = {k: v.to(device) for k, v in encoded.items()}
            
            with torch.no_grad():
                outputs = bert_model(**encoded)
            
            cls_embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            batch_embeddings.append(cls_embedding)
        
        batch_embeddings = np.vstack(batch_embeddings)
        
        # Предсказания для батча
        batch_predictions = model.predict(batch_embeddings)
        
        if hasattr(model, 'predict_proba'):
            batch_probs = model.predict_proba(batch_embeddings)[:, 1]
        else:
            batch_scores = model.decision_function(batch_embeddings)
            batch_probs = 1 / (1 + np.exp(-batch_scores))
        
        predictions.extend(batch_predictions)
        probabilities.extend(batch_probs)
    
    return np.array(predictions), np.array(probabilities)

# Пример пакетного предсказания
print("\nПакетное предсказание для тестовых примеров:")
batch_predictions, batch_probabilities = predict_toxicity_batch(test_examples, final_model, tokenizer, model)

for i, (example, pred, prob) in enumerate(zip(test_examples, batch_predictions, batch_probabilities), 1):
    toxicity_label = "ТОКСИЧНЫЙ" if pred == 1 else "НЕ ТОКСИЧНЫЙ"
    print(f"{i}. {example[:50]}... → {toxicity_label} ({prob:.3f})")


Результаты предсказаний (с использованием Toxic BERT):
--------------------------------------------------------------------------------
1. This is a great article, thanks for sharing!
   → Предсказание: НЕ ТОКСИЧНЫЙ
   → Вероятность токсичности: 0.0012
   → Уверенность: Высокая
--------------------------------------------------------------------------------
2. You are so stupid and worthless, I hate you!
   → Предсказание: ТОКСИЧНЫЙ
   → Вероятность токсичности: 0.9876
   → Уверенность: Высокая
--------------------------------------------------------------------------------
3. I completely disagree with your opinion on this matter.
   → Предсказание: НЕ ТОКСИЧНЫЙ
   → Вероятность токсичности: 0.0012
   → Уверенность: Высокая
--------------------------------------------------------------------------------
4. Go die in a hole, nobody wants you here!
   → Предсказание: ТОКСИЧНЫЙ
   → Вероятность токсичности: 0.9501
   → Уверенность: Высокая
-----------------------------------------------

Вывод: Для проверки работы модели посмотри мкак она класфиицирует новые 10 коментариев, модель уверено справилась с оценкой коментариев по токсичности.

Общий вывод: 1)Имеется датасет с 143 тыс положительных и 16 отрицательных отзывов. Так как я собираюсь использовать bert то не проводил лематизацию и баланс класов ответов.
2) Настроили toxic-bert эмбединги на подготовленых и очищенных данных, для ускорения работы выбрали 1000 строк. Обучили три модели логистическую регрессию, случайный лес и опорные вектора. При помощи крос валидации произвели подбор гиперпаратетров и оценили f1 на тренировочных данных. Лучшей моделью показала себя модельл логистической регрессиив с f1 0.97 на валидации и 0.894 тестовых данных учитывая вес изза дисбаланса класов
3) Для проверки работы модели посмотри мкак она класфиицирует новые 10 коментариев, модель уверено справилась с оценкой коментариев по токсичности.